# 3편: CNN으로 이미지 분류하기

이 노트북에서는 다음을 학습합니다:
- 합성곱(Convolution) 연산의 원리와 FC 레이어와의 차이
- CNN 핵심 구성 요소: Conv2d, BatchNorm2d, ReLU, MaxPool2d, Dropout
- 3-블록 CNN 아키텍처로 FashionMNIST 분류
- 텐서 shape가 각 레이어를 통과하며 어떻게 변하는지 추적
- 특징 맵(Feature Map) 시각화로 CNN 내부 동작 이해
- FC NN vs CNN 성능 비교 (정확도, 학습 시간, 파라미터 수)

## 1. 합성곱(Convolution)이란?

FC(Fully Connected) 레이어는 이미지의 모든 픽셀을 1차원으로 펼쳐서 처리합니다.  
이 과정에서 **인접 픽셀 간의 공간적 관계(지역성)**가 완전히 사라집니다.

합성곱은 작은 **필터(커널)**를 이미지 위에서 슬라이딩하며 특징을 추출합니다:

| 특성 | FC 레이어 | 합성곱 레이어 |
|---|---|---|
| 연결 방식 | 모든 입력 ↔ 모든 출력 | **지역적 연결** (커널 크기만큼) |
| 파라미터 | 입력 × 출력 개수 | **커널 크기 × 채널 수** (공유) |
| 공간 정보 | 1D로 펼쳐서 소실 | **2D 구조 유지** |
| 파라미터 수 | 이미지 크기에 비례 | 이미지 크기와 **무관** |

핵심 원리:
- **지역적 연결(Local Connectivity)**: 전체가 아닌 작은 영역(예: 5×5, 3×3)만 연결
- **파라미터 공유(Parameter Sharing)**: 동일한 필터를 이미지 전체에 재사용
- **결과**: 파라미터 수가 이미지 크기와 무관하게 대폭 감소!

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time

# FashionMNIST 데이터 준비
training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor()
)

batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용 디바이스: {device}")
print(f"학습 데이터: {len(training_data):,}장")
print(f"테스트 데이터: {len(test_data):,}장")
print(f"배치 크기: {batch_size}")

## 2. CNN 핵심 구성 요소

이번 CNN에서 사용할 주요 레이어를 정리합니다:

| 레이어 | 역할 | 이번 모델에서의 사용 |
|---|---|---|
| `Conv2d` | 합성곱 연산으로 특징 추출 (커널이 이미지 위를 슬라이딩) | 5×5, 3×3 커널 |
| `BatchNorm2d` | 각 채널별로 정규화하여 학습 안정화 및 가속 | 모든 Conv 뒤에 적용 |
| `ReLU` | 비선형성 추가 (음수 → 0, 양수 → 그대로) | 활성화 함수 |
| `MaxPool2d` | 공간 크기를 절반으로 축소 (2×2 영역에서 최대값 추출) | Block 1, 2에서 사용 |
| `Dropout` | 학습 중 일부 뉴런을 무작위 비활성화 (과적합 방지) | Classifier에서 사용 |
| `Linear` | 최종 분류를 위한 완전연결 레이어 | 3136→128→10 |

`BatchNorm2d`는 이전 노트북에서는 사용하지 않았던 새로운 요소입니다.  
각 미니배치에 대해 채널별 평균/분산을 정규화하여 **학습 속도 향상**과 **안정적 수렴**에 기여합니다.

In [ ]:
# Shape 추적: 텐서가 각 레이어를 통과하며 어떻게 변하는지 확인
sample = torch.randn(1, 1, 28, 28)
print(f"입력:               {sample.shape}  ← (배치, 채널, 높이, 너비)")
print("=" * 60)

# Block 1: Conv(1→16, 5×5, padding=2) + MaxPool(2)
print("\n[Block 1]")
x = nn.Conv2d(1, 16, kernel_size=5, padding=2)(sample)
print(f"Conv2d(1→16, 5×5):  {x.shape}  ← padding=2로 크기 유지")
x = nn.BatchNorm2d(16)(x)
print(f"BatchNorm2d(16):    {x.shape}  ← 채널별 정규화 (shape 불변)")
x = F.relu(x)
print(f"ReLU:               {x.shape}  ← 활성화 (shape 불변)")
x = F.max_pool2d(x, 2)
print(f"MaxPool2d(2):       {x.shape}  ← 28/2 = 14")

# Block 2: Conv(16→32, 3×3, padding=1) + MaxPool(2)
print("\n[Block 2]")
x = nn.Conv2d(16, 32, kernel_size=3, padding=1)(x)
print(f"Conv2d(16→32, 3×3): {x.shape}  ← padding=1로 크기 유지")
x = nn.BatchNorm2d(32)(x)
print(f"BatchNorm2d(32):    {x.shape}")
x = F.relu(x)
print(f"ReLU:               {x.shape}")
x = F.max_pool2d(x, 2)
print(f"MaxPool2d(2):       {x.shape}  ← 14/2 = 7")

# Block 3: Conv(32→64, 3×3, padding=1) (풀링 없음)
print("\n[Block 3]")
x = nn.Conv2d(32, 64, kernel_size=3, padding=1)(x)
print(f"Conv2d(32→64, 3×3): {x.shape}  ← padding=1로 크기 유지")
x = nn.BatchNorm2d(64)(x)
print(f"BatchNorm2d(64):    {x.shape}")
x = F.relu(x)
print(f"ReLU:               {x.shape}")

# Flatten → Classifier
print("\n[Classifier]")
flat = x.view(1, -1)
print(f"Flatten:            {flat.shape}  ← 64 × 7 × 7 = 3,136")
out = nn.Linear(64 * 7 * 7, 128)(flat)
print(f"Linear(3136→128):   {out.shape}")
out = nn.Linear(128, 10)(out)
print(f"Linear(128→10):     {out.shape}  ← 10개 클래스 점수")

## 3. CNN 모델 구현

3개의 합성곱 블록과 분류기(Classifier)로 구성된 CNN을 설계합니다:

```
입력 (1, 28, 28)
  ┌─────────────────────────────────────────────┐
  │ Block 1: Conv(1→16, 5×5) + BN + ReLU + Pool │ → (16, 14, 14)
  ├─────────────────────────────────────────────┤
  │ Block 2: Conv(16→32, 3×3) + BN + ReLU + Pool│ → (32, 7, 7)
  ├─────────────────────────────────────────────┤
  │ Block 3: Conv(32→64, 3×3) + BN + ReLU       │ → (64, 7, 7)
  └─────────────────────────────────────────────┘
  Flatten → (3136)
  ┌─────────────────────────────────────────────┐
  │ Classifier: Dropout(0.3) + Linear(3136→128) │
  │           + ReLU + Dropout(0.5)              │
  │           + Linear(128→10)                   │ → (10)
  └─────────────────────────────────────────────┘
```

설계 포인트:
- **3개 Conv 블록**: 점진적으로 채널 수 증가 (1→16→32→64)
- **BatchNorm2d**: 모든 Conv 뒤에 적용하여 학습 안정화
- **첫 번째 블록에 5×5 커널**: 넓은 수용 영역(receptive field)으로 저수준 특징 포착
- **padding 사용**: 합성곱 후 공간 크기가 줄어들지 않도록 제어
- **LogSoftmax 없이 CrossEntropyLoss 직접 사용**: PyTorch 권장 패턴

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Block 1: Conv(1→16, 5×5) + BN + ReLU + MaxPool(2)
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),  # (16, 28, 28)
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),  # (16, 14, 14)
        )
        # Block 2: Conv(16→32, 3×3) + BN + ReLU + MaxPool(2)
        self.block2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # (32, 14, 14)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # (32, 7, 7)
        )
        # Block 3: Conv(32→64, 3×3) + BN + ReLU
        self.block3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # (64, 7, 7)
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        # Classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10),
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


cnn_net = ConvNet().to(device)
print(cnn_net)

# 전체 파라미터 수
cnn_params = sum(p.numel() for p in cnn_net.parameters())
print(f"\nCNN 전체 파라미터 수: {cnn_params:,}개")

# 블록별 파라미터 분석
print(f"\n{'블록':15s} {'파라미터 수':>12s}")
print("-" * 30)
for name, module in [("Block 1", cnn_net.block1),
                      ("Block 2", cnn_net.block2),
                      ("Block 3", cnn_net.block3),
                      ("Classifier", cnn_net.classifier)]:
    params = sum(p.numel() for p in module.parameters())
    print(f"{name:15s} {params:>12,}")

In [ ]:
# 데이터 shape 확인
for images, labels in train_dataloader:
    print(f"배치 이미지 shape: {images.shape}")
    print(f"배치 레이블 shape: {labels.shape}")
    print(f"\n→ CNN은 (N, C, H, W) 형태 그대로 사용 (Flatten 불필요)")
    print(f"  FC NN: {images.shape} → view(-1, 784) → (64, 784) 로 펼쳐야 함")
    print(f"  CNN:   {images.shape} → 그대로 입력!")
    break

## 4. CNN 학습

- **옵티마이저**: Adam (적응적 학습률, SGD보다 빠른 수렴)
- **손실 함수**: CrossEntropyLoss (모델에 LogSoftmax 없이 직접 사용)
- **에포크**: 15회

In [ ]:
optimizer_cnn = torch.optim.Adam(cnn_net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().to(device)

num_epochs = 15
cnn_costs = []
cnn_times = []

for epoch in range(num_epochs):
    cnn_net.train()
    start_time = time.time()
    avg_cost = 0
    total_batch = len(train_dataloader)

    for X, Y in train_dataloader:
        X = X.to(device)  # (N, 1, 28, 28) - Flatten 하지 않음!
        Y = Y.to(device)

        optimizer_cnn.zero_grad()
        hypothesis = cnn_net(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer_cnn.step()

        avg_cost += cost.item() / total_batch

    elapsed = time.time() - start_time
    cnn_costs.append(avg_cost)
    cnn_times.append(elapsed)
    print(f"Epoch {epoch+1:02d}/{num_epochs} | cost = {avg_cost:.6f} | time = {elapsed:.2f}s")

print(f"\n평균 에포크 시간: {sum(cnn_times)/len(cnn_times):.2f}s")

## 5. 특징 맵(Feature Map) 시각화

CNN의 각 블록이 이미지에서 어떤 특징을 추출하는지 시각적으로 확인합니다.

- **Block 1**: 엣지, 윤곽선 등 저수준 특징
- **Block 2**: 텍스처, 패턴 등 중간 수준 특징
- **Block 3**: 부분 형태, 의미적 특징 등 고수준 특징

블록이 깊어질수록 더 추상적인 특징을 학습하게 됩니다.

In [ ]:
cnn_net.eval()
sample_img = test_data[0][0].unsqueeze(0).to(device)
sample_label = test_data[0][1]

with torch.no_grad():
    feat1 = cnn_net.block1(sample_img)
    feat2 = cnn_net.block2(feat1)
    feat3 = cnn_net.block3(feat2)

print(f"원본 이미지: {class_names[sample_label]}")
print(f"Block 1 출력: {feat1.shape} (16채널, 14×14)")
print(f"Block 2 출력: {feat2.shape} (32채널, 7×7)")
print(f"Block 3 출력: {feat3.shape} (64채널, 7×7)")

fig, axes = plt.subplots(4, 8, figsize=(16, 8))
fig.suptitle(f"Feature Maps — 입력: {class_names[sample_label]}", fontsize=14)

# 원본 이미지 (첫 번째 행 왼쪽에 표시)
axes[0, 0].imshow(sample_img[0, 0].cpu(), cmap="gray")
axes[0, 0].set_title("Original", fontsize=9)
axes[0, 0].axis("off")
for i in range(1, 8):
    axes[0, i].axis("off")

# Block 1 feature maps (8개)
for i in range(8):
    axes[1, i].imshow(feat1[0, i].cpu(), cmap="viridis")
    axes[1, i].axis("off")
    if i == 0:
        axes[1, i].set_ylabel("Block 1\n(14×14)", fontsize=10)

# Block 2 feature maps (8개)
for i in range(8):
    axes[2, i].imshow(feat2[0, i].cpu(), cmap="viridis")
    axes[2, i].axis("off")
    if i == 0:
        axes[2, i].set_ylabel("Block 2\n(7×7)", fontsize=10)

# Block 3 feature maps (8개)
for i in range(8):
    axes[3, i].imshow(feat3[0, i].cpu(), cmap="viridis")
    axes[3, i].axis("off")
    if i == 0:
        axes[3, i].set_ylabel("Block 3\n(7×7)", fontsize=10)

plt.tight_layout()
plt.show()

## 6. FC NN vs CNN 비교

2편에서 학습한 FC NN을 동일한 조건(Adam, 15 에포크)으로 재학습하여 CNN과 공정하게 비교합니다.

- **FC 1-Layer**: 784 → 10 (단순 선형 분류)
- **FC Multi-Layer**: 784 → 512 → 256 → 10 (3계층 분류기)
- **CNN (3-Block)**: Conv 3블록 + Classifier

In [ ]:
# FC 1-Layer 모델
class SingleLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return self.linear(x)


# FC Multi-Layer 모델
class MultiLayerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )

    def forward(self, x):
        return self.layers(x)


# --- FC 1-Layer 학습 ---
fc1_net = SingleLayerNet().to(device)
fc1_opt = torch.optim.Adam(fc1_net.parameters(), lr=0.001)
fc1_costs, fc1_times = [], []

print("FC 1-Layer 학습 중...")
for epoch in range(num_epochs):
    fc1_net.train()
    start_time = time.time()
    avg_cost = 0
    for X, Y in train_dataloader:
        X, Y = X.view(-1, 784).to(device), Y.to(device)
        fc1_opt.zero_grad()
        cost = criterion(fc1_net(X), Y)
        cost.backward()
        fc1_opt.step()
        avg_cost += cost.item() / len(train_dataloader)
    fc1_costs.append(avg_cost)
    fc1_times.append(time.time() - start_time)
print(f"  완료 — 최종 cost: {fc1_costs[-1]:.6f}")

# --- FC Multi-Layer 학습 ---
fcm_net = MultiLayerNet().to(device)
fcm_opt = torch.optim.Adam(fcm_net.parameters(), lr=0.001)
fcm_costs, fcm_times = [], []

print("FC Multi-Layer 학습 중...")
for epoch in range(num_epochs):
    fcm_net.train()
    start_time = time.time()
    avg_cost = 0
    for X, Y in train_dataloader:
        X, Y = X.view(-1, 784).to(device), Y.to(device)
        fcm_opt.zero_grad()
        cost = criterion(fcm_net(X), Y)
        cost.backward()
        fcm_opt.step()
        avg_cost += cost.item() / len(train_dataloader)
    fcm_costs.append(avg_cost)
    fcm_times.append(time.time() - start_time)
print(f"  완료 — 최종 cost: {fcm_costs[-1]:.6f}")

print("\nFC 1-Layer / FC Multi-Layer / CNN 학습 완료!")

In [ ]:
# --- 테스트 정확도 평가 ---
def evaluate_model(model, dataloader, flatten=False):
    """모델의 전체 정확도와 클래스별 정확도를 계산합니다."""
    model.eval()
    correct = 0
    total = 0
    class_correct = [0] * 10
    class_total = [0] * 10

    with torch.no_grad():
        for X, Y in dataloader:
            if flatten:
                X = X.view(-1, 784)
            X, Y = X.to(device), Y.to(device)
            outputs = model(X)
            _, predicted = torch.max(outputs, 1)
            total += Y.size(0)
            correct += (predicted == Y).sum().item()
            for i in range(Y.size(0)):
                label = Y[i].item()
                class_correct[label] += (predicted[i] == label).item()
                class_total[label] += 1

    overall_acc = 100 * correct / total
    class_acc = {
        class_names[i]: 100 * class_correct[i] / class_total[i]
        for i in range(10)
    }
    return overall_acc, class_acc


fc1_acc, fc1_class_acc = evaluate_model(fc1_net, test_dataloader, flatten=True)
fcm_acc, fcm_class_acc = evaluate_model(fcm_net, test_dataloader, flatten=True)
cnn_acc, cnn_class_acc = evaluate_model(cnn_net, test_dataloader, flatten=False)

# 클래스별 정확도 출력 (CNN)
print("CNN 클래스별 테스트 정확도:")
print("-" * 35)
for name, acc in cnn_class_acc.items():
    bar = "#" * int(acc / 2)
    print(f"  {name:15s} {acc:5.1f}% {bar}")
print(f"\n  {'전체 정확도':15s} {cnn_acc:5.1f}%")

# --- 비교 차트 ---
fc1_params = sum(p.numel() for p in fc1_net.parameters())
fcm_params = sum(p.numel() for p in fcm_net.parameters())

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
epochs = range(1, num_epochs + 1)

# (1) Cost 비교
axes[0].plot(epochs, fc1_costs, "o-", label=f"FC 1-Layer", markersize=3)
axes[0].plot(epochs, fcm_costs, "s-", label=f"FC Multi-Layer", markersize=3)
axes[0].plot(epochs, cnn_costs, "^-", label=f"CNN (3-Block)", markersize=3)
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Cost")
axes[0].set_title("Training Cost")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# (2) 에포크별 학습 시간
axes[1].plot(epochs, fc1_times, "o-", label="FC 1-Layer", markersize=3)
axes[1].plot(epochs, fcm_times, "s-", label="FC Multi-Layer", markersize=3)
axes[1].plot(epochs, cnn_times, "^-", label="CNN (3-Block)", markersize=3)
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Time (seconds)")
axes[1].set_title("Training Time per Epoch")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# (3) 테스트 정확도 비교 (막대 그래프)
model_names = ["FC 1-Layer", "FC Multi-Layer", "CNN (3-Block)"]
accuracies = [fc1_acc, fcm_acc, cnn_acc]
colors = ["#4ECDC4", "#45B7D1", "#FF6B6B"]
bars = axes[2].bar(model_names, accuracies, color=colors, edgecolor="black", linewidth=0.5)
axes[2].set_ylabel("Test Accuracy (%)")
axes[2].set_title("Test Accuracy Comparison")
axes[2].set_ylim(70, 100)
axes[2].grid(True, alpha=0.3, axis="y")
for bar, acc in zip(bars, accuracies):
    axes[2].text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.3,
                 f"{acc:.1f}%", ha="center", va="bottom", fontweight="bold")

plt.tight_layout()
plt.show()

# --- 결과 요약 테이블 ---
print(f"\n{'모델':18s} {'파라미터':>12s} {'평균 시간(s)':>12s} {'최종 Cost':>12s} {'테스트 정확도':>12s}")
print("=" * 72)
print(f"{'FC 1-Layer':18s} {fc1_params:>12,} {sum(fc1_times)/num_epochs:>12.2f} {fc1_costs[-1]:>12.6f} {fc1_acc:>11.1f}%")
print(f"{'FC Multi-Layer':18s} {fcm_params:>12,} {sum(fcm_times)/num_epochs:>12.2f} {fcm_costs[-1]:>12.6f} {fcm_acc:>11.1f}%")
print(f"{'CNN (3-Block)':18s} {cnn_params:>12,} {sum(cnn_times)/num_epochs:>12.2f} {cnn_costs[-1]:>12.6f} {cnn_acc:>11.1f}%")
print()
print(f"CNN vs FC Multi-Layer:")
print(f"  파라미터: {cnn_params:,} vs {fcm_params:,} ({cnn_params/fcm_params*100:.1f}%)")
print(f"  정확도:   {cnn_acc:.1f}% vs {fcm_acc:.1f}% (차이: {cnn_acc - fcm_acc:+.1f}%p)")

## CNN이 효과적인 이유

| 특성 | FC (완전연결) | CNN (합성곱) |
|---|---|---|
| 입력 처리 | 모든 픽셀을 1차원으로 펼침 | 2D 공간 구조를 유지 |
| 연결 방식 | 모든 입출력이 연결 | 커널 크기만큼 지역 연결 |
| 파라미터 규모 | 입력 × 출력 (이미지 커질수록 폭발) | 커널 크기 × 채널 수 (이미지 크기 무관) |
| 특징 학습 | 전역적 패턴만 학습 | 계층적 특징 학습 |

CNN이 이미지에 강한 3가지 핵심 이유:

1. **지역성(Locality)**: 이미지의 의미 있는 패턴은 인접 픽셀에서 나옵니다. CNN의 작은 커널이 이를 효과적으로 포착합니다.

2. **이동 불변성(Translation Invariance)**: 동일한 필터가 이미지 전체를 스캔하므로, 패턴이 어디에 있든 동일하게 감지합니다.

3. **계층적 특징(Hierarchical Features)**: 블록이 깊어질수록 추상적인 특징을 학습합니다.
   - Block 1: 엣지, 코너, 텍스처
   - Block 2: 패턴, 부분 형태
   - Block 3: 객체 형태, 의미적 특징

---

다음 노트북에서는 **GAN(생성적 적대 신경망)**으로 이미지를 '분류'하는 것을 넘어 '생성'하는 방법을 학습합니다.